# trixi PyTorch Example

This example aims to show some basic logging/visualization features for a simple workflow in PyTorch.  
It assumes you have a visdom server running (using port 8080, so you can in principle even connect to the visdom server from some remote device):  
`python -m visdom.server -port 8080`  
Navigate your browser to `localhost:8080` to view the visualizations.


In [1]:
from trixi.logger import PytorchExperimentLogger
from trixi.logger import PytorchVisdomLogger
from trixi.util import Config

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms

Using torch multi processing


Let's start by creating a `PytorchExperimentLogger` and a `PytorchVisdomLogger`.

In [2]:
Exp = PytorchExperimentLogger(base_dir="./experiment_dir", 
                              experiment_name="test-experiment",
                              folder_format="{experiment_name}")

Viz = PytorchVisdomLogger(name="main", port=8080)

The experiment logger creates a convenient folder structure for a given experiment (with name `experiment_name`) in the base directory `base_dir`. To better keep track of when a given experiment was done, `folder_format` allows using [Python's strftime directives](http://strftime.org/). An example folder format using the current date would be:  
`folder_format="%Y-%m-%d_%H-%M_{experiment_name}"`

For the sake of this basic example we'll stick to the simple folder format only using the experiment's name.

In [3]:
!ls experiment_dir

test-experiment


We see the base directory has been created containing our test experiment.

In [4]:
!ls experiment_dir/test-experiment/

checkpoint  config  img  log  plot  result  save


Let's now create a config for our experiment which basically works like a `dict`.

In [5]:
config_dict = Config()

config_dict.batch_size = 64
config_dict.batch_size_test = 1000
config_dict.n_epochs = 10
config_dict.learning_rate = 0.01
config_dict.momentum = 0.9
if torch.cuda.is_available():
    config_dict.use_cuda = True
else:
    config_dict.use_cuda = False
config_dict.rnd_seed = 1
config_dict.log_interval = 200

Time to save this config.

In [6]:
Exp.save_config(config_dict, name="test-config")

The experiment logger's `save_config` method automatically encodes our config in JSON and saves the file in our config directory.

In [7]:
!ls experiment_dir/test-experiment/config/

test-config.json


In [8]:
!tail -n 10 experiment_dir/test-experiment/config/test-config.json

{
    "batch_size": 64,
    "batch_size_test": 1000,
    "n_epochs": 10,
    "learning_rate": 0.01,
    "momentum": 0.9,
    "use_cuda": true,
    "rnd_seed": 1,
    "log_interval": 200
}

Now let's continue with our PyTorch workflow - in this case a simple ConvNet trained on MNIST.

In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if config_dict.use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('experiment_dir/data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=config_dict.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('experiment_dir/data', train=False, transform=transforms.ToTensor()),
    batch_size=config_dict.batch_size_test, shuffle=True, **kwargs)

Processing...
Done!


In [10]:
# build a simple cnn model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [11]:
model = Net()

if config_dict.use_cuda:
    model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=config_dict.learning_rate,
                     momentum=config_dict.momentum)

Now during training we'd like to have a simple visualization of the training loss and every `log_interval` batches we also want to save our model.  
Our visdom logger's `show_value` simply creates a line plot which is automatically appended with new values every iteration. We could also set an appendix to the environment name here so the plot is displayed in a different environment on the visdom server, but we'll stick to the default here.  
Using our experiment logger we can simply save the model providing a name and iteration number. The files are automatically saved in the checkpoint directory of our experiment.

In [12]:
# train
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if config_dict.use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        # plot the training loss
        Viz.show_value(loss.item(), name='Training Loss')
        loss.backward()
        optimizer.step()
        if batch_idx % config_dict.log_interval == 0:
            print('Train Epoch: {} [{}/{} samples ({:.0f}%)]\t Batch Loss: {:.6f}'
                  .format(epoch, batch_idx * len(data), 
                          len(train_loader.dataset),
                          100. * batch_idx / len(train_loader), 
                          loss.item()))
            # save the current model weights
            Exp.save_model(model, name="MNIST_ConvNet", n_iter=batch_idx)

For every epoch we also want to keep track of our model's performance on the test set. Again we can use `show_value` to display a line plot of the test loss for every epoch.

In [13]:
# test
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if config_dict.use_cuda:
            data, target = data.cuda(), target.cuda()
        # data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    # plot the test loss
    Viz.show_value(test_loss, name='Test Loss')
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Let's go ahead and train our net for the number of epochs saved in the config.

In [ ]:
for epoch in range(1, config_dict.n_epochs+1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 samples (0%)]	 Batch Loss: 2.303611
Train Epoch: 1 [12800/60000 samples (21%)]	 Batch Loss: 1.022778
Train Epoch: 1 [25600/60000 samples (43%)]	 Batch Loss: 0.678959
Train Epoch: 1 [38400/60000 samples (64%)]	 Batch Loss: 0.295649
Train Epoch: 1 [51200/60000 samples (85%)]	 Batch Loss: 0.440273

Test set: Average loss: 0.1412, Accuracy: 9597/10000 (95%)

Train Epoch: 2 [0/60000 samples (0%)]	 Batch Loss: 0.287087
Train Epoch: 2 [12800/60000 samples (21%)]	 Batch Loss: 0.351929
Train Epoch: 2 [25600/60000 samples (43%)]	 Batch Loss: 0.291309
Train Epoch: 2 [38400/60000 samples (64%)]	 Batch Loss: 0.254067
Train Epoch: 2 [51200/60000 samples (85%)]	 Batch Loss: 0.247955

Test set: Average loss: 0.0861, Accuracy: 9739/10000 (97%)

Train Epoch: 3 [0/60000 samples (0%)]	 Batch Loss: 0.153974
Train Epoch: 3 [12800/60000 samples (21%)]	 Batch Loss: 0.221300
Train Epoch: 3 [25600/60000 samples (43%)]	 Batch Loss: 0.312968
Train Epoch: 3 [38400/60000 samples (64%)]	 Batc

Now we should also see our saved model checkpoints.

In [ ]:
!ls experiment_dir/test-experiment/checkpoint/

Great! That's it for a very basic use-case, going forward one could e.g. combine the two loggers used in this examples using the `CombinedLogger` class. The benefit of this lies in methods supported by both parts of the combined logger. This way, for example, plots can simultaneously be displayed on the visdom server and saved in the image directory of the experiment - convenient, right?